### Group-by apply

In [214]:
import pandas as pd
import numpy as np

In [215]:
pd_JH_data=pd.read_csv('../data/processed/COVID_relational_confirmed.csv',sep=',',parse_dates=[0])
pd_JH_data=pd_JH_data.sort_values('date',ascending=True).reset_index(drop=True).copy()
pd_JH_data.head()

,date,state,country,confirmed
0,2020-01-22,Alberta,Canada,0.0
1,2020-01-22,no,"Korea, South",1.0
2,2020-01-22,no,Kosovo,0.0
3,2020-01-22,no,Kuwait,0.0
4,2020-01-22,no,Kyrgyzstan,0.0


### Test Data

In [216]:
test_data=pd_JH_data[((pd_JH_data['country']=='US')|
                      (pd_JH_data['country']=='Germany'))&
                     (pd_JH_data['date']>'2020-03-20')]

In [217]:
test_data

,date,state,country,confirmed
15736,2020-03-21,no,Germany,22213.0
15776,2020-03-21,no,US,25818.0
16003,2020-03-22,no,Germany,24873.0
16043,2020-03-22,no,US,33756.0
16268,2020-03-23,no,Germany,29056.0
...,...,...,...,...
39450,2020-06-18,no,US,2191052.0
39676,2020-06-19,no,Germany,190299.0
39716,2020-06-19,no,US,2222579.0
39943,2020-06-20,no,Germany,190670.0


In [218]:
test_data.groupby(['country']).agg(np.max)

,date,state,confirmed
country,,,
Germany,2020-06-20,no,190670.0
US,2020-06-20,no,2255119.0


In [219]:
# %load ../src/features/build_features.py

import numpy as np
from sklearn import linear_model
import pandas as pd

from scipy import signal

reg = linear_model.LinearRegression(fit_intercept=True)

def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate

        Parameters:
        ----------
        in_array : pandas.series

        Returns:
        ----------
        Doubling rate: double
    '''

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope

if __name__ == '__main__':
    test_data_reg=np.array([2,4,6])
    result=get_doubling_time_via_regression(test_data_reg)
    print('the test slope is: '+str(result))

   


the test slope is: [2.]


In [220]:
test_data.groupby(['state','country']).agg(np.max)

date  confirmed
state country                      
no    Germany 2020-06-20   190670.0
      US      2020-06-20  2255119.0

In [221]:
#test_data.groupby(['state', 'country']).apply(get_doubling_time_via_regression)

In [222]:
def rolling_reg(df_input,col='confirmed'):
    ''' input has to be a data frame'''
    ''' return is single series (mandatory for group by apply)'''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)
    return result

In [223]:
test_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed')

state  country       
no     Germany  15736          NaN
                16003          NaN
                16268     7.417994
                16533     7.142035
                16800     8.012983
                           ...    
       US       38920    96.868622
                39185    86.811830
                39450    81.169683
                39716    73.953246
                39984    69.393500
Name: confirmed, Length: 184, dtype: float64

In [224]:
test_data

,date,state,country,confirmed
15736,2020-03-21,no,Germany,22213.0
15776,2020-03-21,no,US,25818.0
16003,2020-03-22,no,Germany,24873.0
16043,2020-03-22,no,US,33756.0
16268,2020-03-23,no,Germany,29056.0
...,...,...,...,...
39450,2020-06-18,no,US,2191052.0
39676,2020-06-19,no,Germany,190299.0
39716,2020-06-19,no,US,2222579.0
39943,2020-06-20,no,Germany,190670.0


In [225]:
pd_DR_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed').reset_index()

In [226]:
pd_DR_result

,state,country,level_2,confirmed
0,Alberta,Canada,0,NaN
1,Alberta,Canada,435,NaN
2,Alberta,Canada,700,NaN
3,Alberta,Canada,967,NaN
4,Alberta,Canada,1263,NaN
...,...,...,...,...
40161,no,Zimbabwe,38900,96.750000
40162,no,Zimbabwe,39167,56.142857
40163,no,Zimbabwe,39432,11.620370
40164,no,Zimbabwe,39698,11.478632


In [227]:
pd_DR_result=pd_DR_result.rename(columns={'confirmed':'confirmed_DR',
                             'level_2':'index'})
pd_DR_result.head()

,state,country,index,confirmed_DR
0,Alberta,Canada,0,NaN
1,Alberta,Canada,435,NaN
2,Alberta,Canada,700,NaN
3,Alberta,Canada,967,NaN
4,Alberta,Canada,1263,NaN


In [228]:
pd_JH_data=pd_JH_data.reset_index()
pd_JH_data.head()

,index,date,state,country,confirmed
0,0,2020-01-22,Alberta,Canada,0.0
1,1,2020-01-22,no,"Korea, South",1.0
2,2,2020-01-22,no,Kosovo,0.0
3,3,2020-01-22,no,Kuwait,0.0
4,4,2020-01-22,no,Kyrgyzstan,0.0


### confirmed_DR == doubling_rate

In [238]:
pd_result_larg=pd.merge(pd_JH_data,pd_DR_result[['index','confirmed_DR']],on=['index'],how='left')
pd_result_larg.head()

,index,date,state,country,confirmed,confirmed_DR
0,0,2020-01-22,Alberta,Canada,0.0,NaN
1,1,2020-01-22,no,"Korea, South",1.0,NaN
2,2,2020-01-22,no,Kosovo,0.0,NaN
3,3,2020-01-22,no,Kuwait,0.0,NaN
4,4,2020-01-22,no,Kyrgyzstan,0.0,NaN


## Filtering the data with groupby apply 


In [239]:
from scipy import signal

def savgol_filter(df_input,column='confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function 
        it ensures that the data structure is kept'''
    window=5, 
    degree=1
    df_result=df_input
    
    filter_in=df_input[column].fillna(0) # attention with the neutral element here
    
    result=signal.savgol_filter(np.array(filter_in),
                           5, # window size used for filtering
                           1)
    df_result[column+'_filtered']=result
    return df_result
        

In [240]:
pd_filtered_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(savgol_filter).reset_index()

In [241]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_result[['index','confirmed_filtered']],on=['index'],how='left')
pd_result_larg.head()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered
0,0,2020-01-22,Alberta,Canada,0.0,NaN,0.0
1,1,2020-01-22,no,"Korea, South",1.0,NaN,0.8
2,2,2020-01-22,no,Kosovo,0.0,NaN,0.0
3,3,2020-01-22,no,Kuwait,0.0,NaN,0.0
4,4,2020-01-22,no,Kyrgyzstan,0.0,NaN,0.0


## Filtered doubling rate

### doubling_rate_filtered = confirmed_filtered_DR

In [249]:

pd_filtered_doubling=pd_result_larg[['state','country','confirmed_filtered']].groupby(['state','country']).apply(rolling_reg,'confirmed_filtered').reset_index()

pd_filtered_doubling=pd_filtered_doubling.rename(columns={'confirmed_filtered':'confirmed_filtered_DR',
                             'level_2':'index'})

pd_filtered_doubling

,state,country,index,confirmed_filtered_DR
0,Alberta,Canada,0,NaN
1,Alberta,Canada,435,NaN
2,Alberta,Canada,700,NaN
3,Alberta,Canada,967,NaN
4,Alberta,Canada,1263,NaN
...,...,...,...,...
40161,no,Zimbabwe,38900,23.446465
40162,no,Zimbabwe,39167,19.914614
40163,no,Zimbabwe,39432,22.549645
40164,no,Zimbabwe,39698,20.316591


In [243]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_doubling[['index','confirmed_filtered_DR']],on=['index'],how='left')
pd_result_larg.tail()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
40161,40161,2020-06-20,no,Barbados,97.0,inf,97.0,-3.412884e+15
40162,40162,2020-06-20,no,Belarus,57936.0,89.614803,57952.4,8.905812e+01
40163,40163,2020-06-20,no,Belgium,60550.0,598.594059,60559.0,5.915538e+02
40164,40164,2020-06-20,no,Albania,1891.0,35.708738,1893.0,3.316968e+01
40165,40165,2020-06-20,no,Zimbabwe,479.0,59.208333,493.4,1.842520e+01


In [244]:
mask=pd_result_larg['confirmed']>100
pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN) 


In [246]:
pd_result_larg

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
0,0,2020-01-22,Alberta,Canada,0.0,NaN,0.0,NaN
1,1,2020-01-22,no,"Korea, South",1.0,NaN,0.8,NaN
2,2,2020-01-22,no,Kosovo,0.0,NaN,0.0,NaN
3,3,2020-01-22,no,Kuwait,0.0,NaN,0.0,NaN
4,4,2020-01-22,no,Kyrgyzstan,0.0,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...
40161,40161,2020-06-20,no,Barbados,97.0,inf,97.0,NaN
40162,40162,2020-06-20,no,Belarus,57936.0,89.614803,57952.4,89.058120
40163,40163,2020-06-20,no,Belgium,60550.0,598.594059,60559.0,591.553816
40164,40164,2020-06-20,no,Albania,1891.0,35.708738,1893.0,33.169675


In [247]:
pd_result_larg[pd_result_larg['country']=='Germany'].tail()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
38878,38878,2020-06-16,no,Germany,188252.0,511.763851,188374.6,478.469280
39145,39145,2020-06-17,no,Germany,188604.0,408.198120,188930.8,353.385856
39410,39410,2020-06-18,no,Germany,189817.0,241.394249,189528.4,327.517074
39676,39676,2020-06-19,no,Germany,190299.0,223.685349,190181.5,303.105301
39943,39943,2020-06-20,no,Germany,190670.0,446.100821,190834.6,291.198132


In [248]:
pd_result_larg.to_csv('../data/processed/COVID_final_set.csv',sep=',',index=False)